In [ ]:
import panel as pn
import hvplot.pandas
import holoviews as hv
import xarray as xr
import pandas as pd
import datetime as dt
import numpy as np
from datetime import date
from datetime import datetime
from holoviews import dim, opts
from bokeh.models import HoverTool
import ipywidgets as ipw
import hvplot.xarray
import panel.widgets as pnw
import plotly.express as px
from matplotlib import pyplot as plt
import geoviews as gv
import cartopy.crs as ccrs
import seaborn as sns
from matplotlib import colors as mcolors
import shutup
shutup.please()
hv.extension('bokeh')
pn.extension('tabulator', sizing_mode="stretch_width")

In [ ]:
## Make sure to upload 'Tmax_tmin.nc' to files on the left everytime notebook is opened again
data = xr.open_dataset('Tmax_Tmin.nc')
tmax = data.tmax
tmin = data.tmin
# data

In [ ]:
time = data.time.values.tolist()
city = data.City.values.tolist()
lat = data.latitude.values
lon = data.longitude.values
regions = ['Northeast', 'Midwest', 'South', 'West']

In [ ]:
# Using a dictionary to map a city to its region (South, West, Northeast, Midwest)
city_to_region = {
    'Birmingham': 'South', 'Mobile': 'South', 'Phoenix': 'West', 'Bullhead City': 'West', 'Little Rock': 'South',
    'Bentonville': 'South', 'Los Angeles': 'West', 'Redding': 'West', 'Denver': 'West', 'Colorado Springs': 'West',
    'Bridgeport': 'Northeast', 'Middletown': 'Northeast', 'Wilmington': 'South', 'Dover': 'South', 'Washington': 'Northeast',
    'Jacksonville': 'South', 'Pensacola': 'South', 'Atlanta': 'South', 'Savannah': 'South', 'Boise City': 'West',
    'Pocatello': 'West', 'Chicago': 'Midwest', 'Quincy': 'Northeast', 'Indianapolis': 'Midwest', 'Evansville': 'Midwest',
    'Des Moines': 'Midwest', 'Dubuque': 'Midwest', 'Wichita': 'Midwest', 'Shawnee': 'Midwest', 'Louisville': 'South',
    'Bowling Green': 'South', 'New Orleans': 'South', 'Bossier City': 'South', 'Portland': 'West', 'Baltimore': 'Northeast',
    'Frederick': 'Northeast', 'Boston': 'Northeast', 'Westfield': 'Midwest', 'Detroit': 'Midwest', 'Grand Rapids': 'Midwest',
    'Minneapolis': 'Midwest', 'Rochester': 'Northeast', 'Jackson': 'South', 'Southaven': 'South', 'Kansas City': 'Midwest',
    'Florissant': 'Midwest', 'Billings': 'West', 'Missoula': 'West', 'Omaha': 'Midwest', 'Lincoln': 'Midwest', 'Las Vegas': 'West',
    'Carson City': 'West', 'Manchester': 'Northeast', 'Nashua': 'Northeast', 'Newark': 'Northeast', 'Camden': 'Northeast',
    'Albuquerque': 'West', 'Las Cruces': 'West', 'New York': 'Northeast', 'Buffalo': 'Northeast', 'Charlotte': 'South',
    'Rocky Mount': 'South', 'Fargo': 'Midwest', 'Bismarck': 'Midwest', 'Columbus': 'Midwest', 'Warren': 'Midwest',
    'Oklahoma City': 'South', 'Tulsa': 'South', 'Medford': 'West', 'Philadelphia': 'Northeast', 'Pittsburgh': 'Northeast',
    'Providence': 'Northeast', 'Woonsocket': 'Northeast', 'Columbia': 'South', 'Charleston': 'South', 'Sioux Falls': 'Midwest',
    'Rapid City': 'Midwest', 'Memphis': 'South', 'Johnson City': 'South', 'Houston': 'South', 'Amarillo': 'South',
    'Salt Lake City': 'West', 'Spanish Fork': 'West', 'Burlington': 'Northeast', 'Virginia Beach': 'South', 'Roanoke': 'South',
    'Seattle': 'West', 'Spokane': 'West', 'Huntington': 'Midwest', 'Milwaukee': 'Midwest', 'La Crosse': 'Midwest',
    'Cheyenne': 'West', 'Casper': 'West', 'Toronto': 'Northeast'
}

In [ ]:
df = tmax.to_dataframe().reset_index()
df_tmin = tmin.to_dataframe().reset_index()
df['tmin'] = df_tmin['tmin']
# df = df.loc[:,['City', 'time', 'tmax', 'tmin']]

#removing duplicate 'Portland' column
df = df.loc[:,~df.columns.duplicated()].copy()
df = df.rename(columns={'tmax': 'Max Temp (Fahrenheit)', 'tmin': 'Min Temp (Fahrenheit)'})

#Adding necessary columns for region
df['Region'] = df['City'].map(city_to_region)

#Adding necessary time columns
df['time'] = pd.to_datetime(df['time'])
df['Year-Month'] = df['time'].dt.strftime('%Y-%m')
df['Year'] = df['time'].dt.year.astype('int')
df['Month'] = df['time'].apply(lambda x: x.strftime('%B'))

#Adding columns for the Map visualization
df['Yearly Max'] = df.groupby(['City', 'Year'])['Max Temp (Fahrenheit)'].transform('max')
df['Yearly Min'] = df.groupby(['City', 'Year'])['Max Temp (Fahrenheit)'].transform('min')
df = df.rename(columns = {'latitude': 'Latitude', 'longitude': 'Longitude', 'Max Temp': 'Max Temp (Fahrenheit)', 'Min Temp': 'Min Temp (Fahrenheit)'})

# making DataFrame pipleine interactive
# idf = df.interactive()
# idf.head()

In [ ]:
df.head()

In [ ]:
# #| title: Temperature Data Over Time
# months = df['Month'].unique().tolist()
# #Widget to select multiple cities
# select_city = pn.widgets.MultiChoice(
#     name='Select Cities',
#     options=city,
#     value = ['New York', 'Los Angeles']
# )
# #Widget to toggle max/min temperature
# max_or_min = pn.widgets.RadioBoxGroup(name='Max/Min Temperature', options=['Max Temp (Fahrenheit)', 'Min Temp (Fahrenheit)'])
# #Widget to select a month
# select_month = pn.widgets.Select(
#     name='Select Month',
#     options=months+['All Months'],
#     value = 'August'
# )
# #Widget to select number of years to include
# numyears = pn.widgets.IntRangeSlider(
#     name = "Year Range",
#     start = 1950,
#     end = 2021,
#     value = (2016, 2021),
#     step = 1
# )
# ipipeline_1 = idf[(idf['City'].isin(select_city.param.value)) &
#                     (idf['Year'] >= numyears.param.value_start) &
#                     (idf['Year'] <= numyears.param.value_end)]

# line_plot = ipipeline_1.hvplot(x='time', y=max_or_min, by='City', line_width=0.8,
#                                 height=350, width=1500,
#                                 cmap='Accent', ylim=(-40, 120))


# @pn.depends(select_city.param.value, max_or_min.param.value, numyears.param.value_start, numyears.param.value_end)
# def create_line(cities, max_min, year_start, year_end):
#     constrained = df[(df['City'].isin(cities)) & (df['Year'] >= year_start) &
#                     (df['Year'] <= year_end)]
#     upper = max(constrained[max_min])+5
#     lower = min(constrained[max_min])-5
#     fig = constrained.hvplot(x='time', y=max_or_min, by='City', line_width=0.8,
#                                 height=350, width=1500,
#                                 cmap='Accent', ylim=(-40, 120))
#     return fig
                   
# layout = pn.Column(select_city, max_or_min, numyears, create_line)
# layout.servable()
# layout

In [ ]:
# col_1 = pn.Column(select_city, max_or_min, numyears, create_line)
# col_1

In [ ]:
# plotly graph

df = px.data.gapminder().query("country=='Canada'")
fig = px.line(df, x="year", y="lifeExp", title='Life expectancy in Canada')

In [ ]:
def create_simple_lineplot():
    constrained = df[(df['City']=='New York') & (df['Year'] >= 2010) &
                    (df['Year'] <= 2021)]
    fig = px.line(constrained, x="Year", y='Max Temp (Fahrenheit)')
    return fig
create_simple_lineplot()